In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect(":memory:")
cur = con.cursor()

In [3]:
link = 'https://raw.githubusercontent.com/eric-bunch/boston_housing/master/boston.csv'

In [4]:
data = pd.read_csv(link)
data.rename(columns = {'MDEV':'MEDV'}, inplace=True)

In [5]:
data.to_sql("boston", con, index=False, if_exists="replace")

In [6]:
pd.read_sql(
    """
    SELECT * FROM boston
    limit 5
    """,
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


## Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [22]:
pd.read_sql(
    """
    SELECT
        SUM(CASE WHEN "CRIM" IS NULL THEN 1 ELSE 0 END) AS CRIM,
        SUM(CASE WHEN "ZN" IS NULL THEN 1 ELSE 0 END) AS ZN,
        SUM(CASE WHEN "INDUS" IS NULL THEN 1 ELSE 0 END) AS INDUS,
        SUM(CASE WHEN "CHAS" IS NULL THEN 1 ELSE 0 END) AS CHAS,
        SUM(CASE WHEN "NOX" IS NULL THEN 1 ELSE 0 END) AS NOX
    """,
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX
0,0,0,0,0,0



## Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [29]:
pd.read_sql(
    """
    SELECT 
        COUNT(DISTINCT "CRIM") AS CRIM, COUNT(DISTINCT "ZN") AS ZN, 
        COUNT(DISTINCT "INDUS") AS INDUS, COUNT(DISTINCT "CHAS") AS CHAS, 
        COUNT(DISTINCT "NOX") AS NOX FROM boston
    """,
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX
0,504,26,76,2,81


## Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по данным в этих колонках

In [31]:
pd.read_sql(
    """
    with min_med as (
    SELECT
    min(CRIM) as min_crim,
    (SELECT CRIM as m_crim FROM boston ORDER BY CRIM LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston) / 2) as med_crim,
    min(ZN) as min_zn,
    (SELECT ZN as m_crim FROM boston ORDER BY ZN LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston) / 2) as med_zn,
    min(INDUS) as min_indus,
    (SELECT INDUS as m_crim FROM boston ORDER BY INDUS LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston) / 2) as med_indus,
    min(CHAS) as min_chas,
    (SELECT CHAS as m_crim FROM boston ORDER BY CHAS LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston) / 2) as med_chas,
    min(NOX) as min_nox,
    (SELECT NOX as m_crim FROM boston ORDER BY NOX LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston) / 2) as med_nox,
    min(RM) as min_rm,
    (SELECT RM as m_crim FROM boston ORDER BY RM LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston) / 2) as med_rm,
    min(AGE) as min_age,
    (SELECT AGE as m_crim FROM boston ORDER BY AGE LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston) / 2) as med_age,
    min(DIS) as min_dis,
    (SELECT DIS as m_crim FROM boston ORDER BY DIS LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston) / 2) as med_dis,
    min(RAD) as min_rad,
    (SELECT RAD as m_crim FROM boston ORDER BY RAD LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston) / 2) as med_rad,
    min(TAX) as min_tax,
    (SELECT TAX as m_crim FROM boston ORDER BY TAX LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston) / 2) as med_tax,
    min(PTRATIO) as min_ptratio,
    (SELECT PTRATIO as m_crim FROM boston ORDER BY PTRATIO LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston) / 2) as med_ptratio,
    min(B) as min_b,
    (SELECT B as m_crim FROM boston ORDER BY B LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston) / 2) as med_b,
    min(LSTAT) as min_lstat,
    (SELECT LSTAT as m_crim FROM boston ORDER BY LSTAT LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston) / 2) as med_lstat,
    min(MEDV) as min_medv,
    (SELECT MEDV as m_crim FROM boston ORDER BY MEDV LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston) / 2) as med_medv
    from boston
        limit 5)
    select min_zn, med_zn, min_chas, med_chas from min_med

    """,
    con,
)

,min_zn,med_zn,min_chas,med_chas
0,0.0,0.0,0.0,0.0


## Выведите разницу между среднем количеством комнат(RM) в домах с самой дорогой стоимостью(MEDV) и 25 самыми дешевыми домами. Аналогично по 50, 100, 200, 300 самыми дешевыми домами. (кол-во домов(25,50,100,200,300), среднее кол-во комнат в них, среднее кол-во комнат в самых дорогих, разница). Напишите влияет ли кол-во комнат на стоимость и как сильно.

In [38]:
pd.read_sql(
    """
   with
   lim25 as(
   SELECT COUNT(*) as cnt_house, avg(rm) AS rvmin, (select rm from boston order by medv desc limit 1) AS rvmax
   from (select * from boston limit 25) AS a),

   lim50 as(
   SELECT COUNT(*) as cnt_house, avg(rm) AS rvmin, (select rm from boston order by medv desc limit 1) AS rvmax
   from (select * from boston limit 50) AS a),

   lim100 as(
   SELECT COUNT(*) as cnt_house, avg(rm) AS rvmin, (select rm from boston order by medv desc limit 1) AS rvmax
   from (select * from boston limit 100) AS a),

   lim200 as(
   SELECT COUNT(*) as cnt_house, avg(rm) AS rvmin, (select rm from boston order by medv desc limit 1) AS rvmax
   from (select * from boston limit 200) AS a),

   lim300 as(
   SELECT COUNT(*) as cnt_house, avg(rm) AS rvmin, (select rm from boston order by medv desc limit 1) AS rvmax
   from (select * from boston limit 300) AS a),

   limunion as (
   SELECT * from lim25
   union all
   SELECT * from lim50
   union all
   SELECT * from lim100
   union all
   SELECT * from lim200
   union all
   SELECT * from lim300
   )

    SELECT cnt_house, rvmin, rvmax, (rvmax - rvmin) AS rvrazn from limunion


    """,
    con,
)

,cnt_house,rvmin,rvmax,rvrazn
0,25,6.130040,7.489,1.358960
1,50,6.086760,7.489,1.402240
2,100,6.234410,7.489,1.254590
3,200,6.252390,7.489,1.236610
4,300,6.420333,7.489,1.068667


## Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг). Напишите какой вывод можно сделать по этим данным.

In [34]:
pd.read_sql(
    """
    SELECT * FROM boston
    limit 5
    """,
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


## Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [40]:
pd.read_sql(
    """
    SELECT avg(medv) AS cost, case chas WHEN '1.0' THEN 'Граничит' ELSE 'Не граничит' end AS border_river FROM boston
    group by chas
    limit 50
    """,
    con,
)

,house_cost,border_river
0,22.093843,Не граничит
1,28.440000,Граничит


## Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

In [36]:
pd.read_sql(
    """
    SELECT * FROM boston
    limit 5
    """,
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


## Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [42]:
pd.read_sql(
    """
   with prc as (
   SELECT INDUS, NOX, NTILE(10) OVER(ORDER BY INDUS) as percen FROM boston
   )
   SELECT percen * 10 AS percentile, INDUS, NOX from prc GROUP BY percentile

    """,
    con,
)

,percentile,INDUS,NOX
0,10,0.46,0.422
1,20,2.93,0.401
2,30,4.39,0.442
3,40,5.96,0.499
4,50,7.38,0.493
5,60,9.69,0.585
6,70,13.89,0.550
7,80,18.10,0.693
8,90,18.10,0.740
9,100,19.58,0.871
